# Tests factorization

In [1]:
include("factorizatons.jl")

using LinearAlgebra, Permutations, Random
using QPSReader, QuadraticModels
using BenchmarkTools

A = rand(5,5)
As1 = A*A'
As2 = [[6,12,3.0,-6] [12,-8,-13,4] [3.0,-13,-7,1] [-6,4,1,6]]
As3 = Matrix(Symmetric(rand(MersenneTwister(0),4,4)))
allMatrix = (As1, As2, As3)
for i in 1:3
    As = allMatrix[i]
    # Bunch Parlett
    L, D, perms, loc = bunch_parlett(As)
    P = Matrix(Permutation(perms))
    Asx = P*L*D*L'*P'
    println("Problem $i: BP , error = ", norm(As-Asx))
    # Bunch Kaufmann
    L, D, perms, loc = bunch_kaufmann(As)
    P = Matrix(Permutation(perms))
    Asx = P*L*D*L'*P'
    println("Problem $i: BK , error = ", norm(As-Asx))
    # bounded Bunch Kaufmann
    L, D, perms, loc = bounded_bunch_kaufmann(As)
    P = Matrix(Permutation(perms))
    Asx = P*L*D*L'*P'
    println("Problem $i: BBK , error = ", norm(As-Asx))
    # Aasen
    L,T,perms = aasen(As)
    P = Matrix(Permutation(perms))
    Asx = P*L*T*L'*P'
    println("Problem $i: Aasen , error = ", norm(As-Asx))
end

Problem 1: BP , error = 1.3597399555105182e-16
Problem 1: BK , error = 

2.4196749845665633e-16


Problem 1: BBK , error = 2.4196749845665633e-16
Problem 1: Aasen , error = 

2.5438405243138006e-16
Problem 2: BP , error = 1.2755491433176288e-15
Problem 2: BK , error = 0.0
Problem 2: BBK , error = 9.155133597044475e-16
Problem 2: Aasen , error = 0.0
Problem 3: BP , error = 1.1102230246251565e-16
Problem 3: BK , error = 6.206335383118183e-17
Problem 3: BBK , error = 6.206335383118183e-17
Problem 3: Aasen , error = 1.1102230246251565e-16


# Tests Solver

In [2]:
As0 = [[6,12,3.0,-6] [12,-8,-13,4] [3.0,-13,-7,1] [-6,4,1,6]]
b0 = rand(4)
L1, D1, perms1, loc1 = bunch_kaufmann(As0)
xbk = solve(L1, D1, perms1, loc1, b0)
L2,T2,perms2 = aasen(As0)
xaas = solve(L2,T2,perms2,b0)
x0 = As0\b0
println("error solveur BK: ", norm(xbk-x0))
println("error solveur Aasen: ", norm(xaas-x0))

error solveur BK: 4.5534426050565124e-17
error solveur Aasen: 1.962615573354719e-17


# Benchmark avec Lapack

In [6]:
n = 10
A_bm = rand(n,n)
syst = A_bm*A_bm'
# Bunch Kaufmann
println("Benchmark: Bunch-Kaufmann Lapack vs Projet")
@benchmark bunchkaufman(Symmetric(syst), false)


Benchmark: Bunch-Kaufmann Lapack vs Projet


BenchmarkTools.Trial: 10000 samples with 185 evaluations.
 Range (min … max):  570.043 ns … 35.298 μs  ┊ GC (min … max):  0.00% … 97.30%
 Time  (median):     886.486 ns              ┊ GC (median):     0.00%
 Time  (mean ± σ):     1.230 μs ±  2.678 μs  ┊ GC (mean ± σ):  28.18% ± 12.35%

  █▆                                                            
  ██▂▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▂▂▂▂▂▂▂▂▂▂ ▂
  570 ns          Histogram: frequency by time         19.9 μs <

 Memory estimate: 6.17 KiB, allocs estimate: 6.

In [7]:
@benchmark bunch_kaufmann(syst)


BenchmarkTools.Trial: 10000 samples with 3 evaluations.
 Range (min … max):   8.458 μs …  1.295 ms  ┊ GC (min … max):  0.00% … 98.92%
 Time  (median):      9.000 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   10.716 μs ± 41.491 μs  ┊ GC (mean ± σ):  12.79% ±  3.28%

    ██▃▁                                                       
  ▁▅████▆▅▅▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  8.46 μs         Histogram: frequency by time          15 μs <

 Memory estimate: 25.06 KiB, allocs estimate: 275.

In [8]:
println("Benchmark: Bounded-Bunch-Kaufmann (rook pivoting) Lapack vs Projet")
@benchmark bunchkaufman(Symmetric(syst), true)


Benchmark: Bounded-Bunch-Kaufmann (rook pivoting) Lapack vs Projet


BenchmarkTools.Trial: 10000 samples with 187 evaluations.
 Range (min … max):  561.717 ns … 30.465 μs  ┊ GC (min … max):  0.00% … 97.21%
 Time  (median):     801.249 ns              ┊ GC (median):     0.00%
 Time  (mean ± σ):     1.112 μs ±  2.233 μs  ┊ GC (mean ± σ):  26.33% ± 12.35%

  █▇                                                           ▁
  ██▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▃▅▆▆▆▇ █
  562 ns        Histogram: log(frequency) by time      17.7 μs <

 Memory estimate: 6.17 KiB, allocs estimate: 6.

In [9]:
@benchmark bounded_bunch_kaufmann(syst)

BenchmarkTools.Trial: 10000 samples with 3 evaluations.
 Range (min … max):   8.736 μs …  1.364 ms  ┊ GC (min … max):  0.00% … 98.44%
 Time  (median):      9.292 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   11.095 μs ± 42.431 μs  ┊ GC (mean ± σ):  12.62% ±  3.27%

    █▇▃                                                        
  ▁▆███▇▆▅▅▄▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  8.74 μs         Histogram: frequency by time        15.7 μs <

 Memory estimate: 25.31 KiB, allocs estimate: 282.

# Application: optimisation quadratique